In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

/home/nsml/.conda/envs/fc_peft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "./outputs/lora/checkpoint-8325"
config = PeftConfig.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [3]:
inference_model = PeftModel.from_pretrained(model, model_path)
inference_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 3072)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=3072, out_features=9216, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=9216, bias=False)
                )
     

In [4]:
pipe = pipeline(
    "text-generation",
    model=inference_model,
    tokenizer=tokenizer,
    device=0
)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Pe

In [10]:
def build_prompt(title, doc):
    return f"다음의 문서를 요약하시오\n제목: {title}\n본문: {doc}\n요약:"

In [19]:
title = "일론 머스크, '세계 최고 부자' 타이틀 탈환... 순자산 253조원"

In [20]:
doc = """
일론 머스크(54) CEO가 다시 한번 세계 최고 부자 타이틀을 되찾았다.
앞서 전기차 업체 ‘테슬라’ 주가가 급락하면서 명품 브랜드 그룹 ‘루이뷔통모에헤네시(LVMH)’의 베르나르 아르노(74) 회장에게 내줬던 자리를 다시 탈환한 것이다.
테슬라의 주가가 상승세를 그리면서 머스크 CEO의 순자산은 올해 1월 이후 553억달러(약 72조원) 더 늘어나 1920억달러(약 253조원)를 기록 중이다.
블룸버그가 집계하는 전 세계 억만장자 순위에 따르면 아르노 회장의 재산은 245억달러(약 32조원) 감소한 1870억달러(약 244조원) 에 머물렀다.
"""

In [21]:
pipe(build_prompt(title, doc),
     do_sample=False,
     max_new_tokens=128,
     return_full_text=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': ' 일일론 머스크 CEO가 다시 한번 세계 최고 부자 타이틀을 되찾았는데, 올해 1월 이후 테슬라의 주가가 상승세를 그리면서 머스크 CEO의 순자산은 올해 1월 이후 553억달러 더 늘어나 1920억달러(약 253조원)를 기록 중이며, 아르노 회장의 재산은 245억달러(약 32조원) 감소한 1870억달러(약 244조원) 에 머물렀다.'}]